In [9]:
from nltk.tokenize import TreebankWordTokenizer
tokenizer=TreebankWordTokenizer()
import numpy as np
import re
import math
import pandas as pd

In [2]:
def indexing(sort):
    index=True
    arr=[sort[0]]
    num=[1]
    temp=0                              
    for i in range(1,len(sort),1):
        for j in range(0,len(removal),1):
            if(sort[i]==removal[j]):
                index=False                #조사 제거(a, the, ...)
        if(index==True):
            if(sort[i]==sort[i-1]):
                num[temp]+=1
            else:
                temp+=1
                arr.append(sort[i])
                num.append(1)
        index=True
        
    
    return (arr,num)

    

In [3]:
f = open("C:/Users/tjswn/removal.txt", 'r')
r = f.read()
removal=r.split(',')

In [4]:
movielist=["C:/Users/tjswn/lalaland.txt","C:/Users/tjswn/coco.txt","C:/Users/tjswn/9.txt","C:/Users/tjswn/2012.txt","C:/Users/tjswn/batman.txt","C:/Users/tjswn/big.txt","C:/Users/tjswn/getout.txt","C:/Users/tjswn/it.txt","C:/Users/tjswn/logan.txt","C:/Users/tjswn/next.txt"]
movie=['lalaland','coco','9','2012','batman','big','getout','it','logan','next']

ar=[[],[],[],[],[],[],[],[],[],[]]
nu=[[],[],[],[],[],[],[],[],[],[]]

for k in range (0,len(movielist),1):
    f = open(movielist[k])
    data = f.read()

    data = re.sub('[\'-=.#,!"/&()?:$}\;]', '', data)    #특수문자 제거, ; []는 제거 안됨

    p = re.compile("[^0-9]")                        #숫자 제거
    data="".join(p.findall(data))

    data=data.lower()                               #소문자 전환

    text=tokenizer.tokenize(data)                   #토큰화

    sort=sorted(text)
    
    allnum=np.zeros(len(movielist))
    arr,num=indexing(sort)
    

    ar[k].extend(arr)
    nu[k].extend(num)
    
    f.close()


In [5]:
temp=[]
for i in range(0,len(movielist),1):
    temp.extend(ar[i])
temp = list(set(temp))
temp=sorted(temp)

arrtemp=np.zeros((len(temp),len(movielist)))

for i in range(0,len(movielist),1):
    for j in range(0,len(temp),1):
        try:
            num=ar[i].index(temp[j])
        except ValueError as er:                           #단어가 없는 경우 에러 처리
            arrtemp[j][i]=0
        else:
            arrtemp[j][i]=nu[i][num]

14516


In [6]:
weight=np.zeros((len(temp),len(movielist)))
tf=0
idf=0
maxf=0
for i in range(0,len(temp),1):
    maxf=max(arrtemp[i])
    nonzero=np.count_nonzero(arrtemp[i])
    idf=math.log(len(movielist)/nonzero)
    for j in range(0,len(movielist),1):
        tf=arrtemp[i][j]/maxf
        weight[i][j]=tf*idf
print(weight)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         2.30258509 ... 0.         0.         0.        ]
 [1.60943791 0.         0.         ... 0.         0.         1.60943791]
 ...
 [0.         2.30258509 0.         ... 0.         0.         0.        ]
 [0.         2.30258509 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         2.30258509 0.        ]]


In [7]:
U, s, V = np.linalg.svd(weight, full_matrices = True)
S = np.zeros(weight.shape)
for i in range(len(s)):
    S[i][i] = s[i]
U=(U.T[0:10]).T
S=(S.T).T[0:10]

In [10]:
us=U.dot(S)
sv=S.dot(V)
word='guitar'
num=0
arrnum=np.empty(10)
try:
    num=temp.index(word)
except ValueError as er:                           #단어가 없는 경우 에러 처리
    num=0
else:
    arrnum=us[num]
result=np.empty(10)
result=arrnum.dot(sv)
df=pd.DataFrame({"movielist":movie,"점수":result})
df

,movielist,점수
0,lalaland,3.556406
1,coco,87.014869
2,9,1.086216
3,2012,1.275226
4,batman,2.174198
5,big,0.340527
6,getout,2.068521
7,it,1.925544
8,logan,1.646157
9,next,1.658444


In [11]:
U1=(U.T[0:8]).T
S1=(S.T[0:8]).T[0:8]
V1=(V.T).T[0:8]

In [12]:
sv1=S1.dot(V1)
us1=U1.dot(S1)
word='guitar'
num=0
arrnum=np.empty(8)
try:
    num=temp.index(word)
except ValueError as er:                           #단어가 없는 경우 에러 처리
    num=0
else:
    arrnum=us1[num]
result=np.empty(10)
result=arrnum.dot(sv1)
df=pd.DataFrame({"movielist":movie,"점수":result})
df

,movielist,점수
0,lalaland,3.284686
1,coco,86.864672
2,9,4.512765
3,2012,1.154006
4,batman,2.090075
5,big,0.694745
6,getout,1.572522
7,it,1.786701
8,logan,1.521885
9,next,1.500432
